# Convolutional Neural Networks

## Neurons as Feature Detectors
Recall how a neuron's value is calculated. It consists of an aggregation and an activation step. It takes several real numbers as input and outputs a single real number.
\begin{equation}
Aggregation: Agg=b_1x_1 + b_2x_2 + \cdots + b_nx_n
\end{equation}
\begin{equation}
Activation: y=\sigma(Agg)
\end{equation}
Note that the parameters of this neuron are $b_i$ and are $n$ in number. 
You may recall from JEE math that the aggregation step can be written as a dot product:
\begin{equation}
Agg=b\cdot x= |b||x|cos(\theta)
\end{equation}

![neuron](https://i.imgur.com/2NRhbZy.png)

Where $b$ and $x$ are column vectors and $\theta$ is the angle between them. For fixed magnitudes, the aggregate will be maximised when the angle between them is 0. In other words, the aggregattion step measures the similarity between $x$ and $b$ (specifically the cosine similarity). When the aggregation is large and positive, the neuron's activation will approach 1 while if they are very dissimilar, the neuron should approach 0. Thus, a neuron can be interpreted as a feature detector. Presence of the feature in the input is indicated by the value of the neuron's output.

There is a catch, however. In the preceding section, we assumed that the magnitudes of $x$ and $b$ were fixed. It is reasonable to ignore the magnitude of $b$ since we are only interested in comparing the 'strength' of the feature's presence across different values of $x$. Ignoring the magnitude of $x$ is dubious because we could drive the aggregate to a large value simply by letting all the elements of $x$ be large values. This is why it is important to scale your data before passing it to any learning algorithm.

## Images and Feed Forward Neural Networks

Suppose we want to use a neural network to classify images. An image consists of pixel values arranged in a matrix whose shape matches the resolution of the image. Since each pixel has a separate value for red, green and blue levels, you can imagine them as 3 matrices stacked up (3 channels). In order to use the image as input to the neural network from the last class, we need to transform this data into a 1-D vector. You should notice that the neural network we've been studying so far does not care about the ordering of the input vector, as long as all samples are ordered the same way. This order-independence allows us to squash our image in any way we want as long as we use the same method of squashing for each image in our dataset.

For simplicity, we will use grayscale images, which only require 1 matrix to store pixel values. Our squashing method is placing rows of the image next to each other. 

![fmnist](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png)

The FMNIST dataset contains 60000 images in 10 classes: 

T-shirt/top
, Trouser
, Pullover
, Dress
, Coat
, Sandal
, Shirt
, Sneaker
, Bag
, Ankle boot 


Thinking back to our feature detector discussion, consider a neuron in the first hidden layer. It has 784 inputs, each associated with a weight. The output of the neuron measures the similarity of the 784 inputs to the weights. If we plot these weights in a 28x28 image, we can see what feature the neuron is detecting.
Here are the filters learned for 0 hidden layers. (green=positive, red=negative)

![0 hidden layers](https://i.imgur.com/jh5IN0J.png)

The case of zero hidden layers is equivalent to logistic regression. Let us plot the weights learned for a network with a single hidden layer.

![1 hidden layer](https://i.imgur.com/GnxXohj.png)
We see that the first layer learns many duplicate features

Neurons in subsequent layers can be analysed similarly, however plotting them will not yield very interpretable information. 

![1 hidden layer](https://i.imgur.com/Ru1idV6.png)

These neurons are actually looking for combinations of features from the first layer that result in more abstract representations of the input. For example, the first layer might detect similarity to circles and squares while the next layer could combine be active when both those neurons are active.

## Convolution and Spatial Proximity
In the previous section, we were measuring the similarity of the image to a learned weight matrix. But what would happen if the object we are trying to detect doesn't overlap significantly with the learned weights? What if our object of interest is in the corner of the image? The detector neuron would not be activated and we would not detect the object.

We could instead break the image into smaller pieces and look for parts of the object. This is what the convolution operator attempts to do. In the previous section, we looked to match the image with a 28x28 weight matrix. Why not look at every possible 5x5 square in the image and match it to a learned 'filter' of the same size?

![Convolution operator](https://i.stack.imgur.com/I7DBr.gif)

The calculation we are performing is the same as before. we are simply doing it in patches of the image. The operation of performing this dot product over all patches of the image is known as convolution. Each dot product yields a real number, so the convolution operator outputs a matrix of these scalars.

![Kirsch filter](https://i.imgur.com/FKkZKjY.png)

![Kirsch filter](https://i.imgur.com/ildbL7Y.png)

Above is the output of the 8 Kirsch filters that find edges oriented along the compass directions.


One advantage convolution offers over a feed forward network is that it greatly reduces the parameters to be learned. We have gone from learning a 28x28 filter to learning a 5x5 filter for a single neuron. That amounts to a reduction by a factor of 30. The disadvantage is that this filter, being so small, cannot capture very complex information about the image. At best we can find edges in the first layer. As before, the network will detect more complex features in the deeper layers. It could use a vertical edge and a horizontal edge to determine whether there is a T shape in the image.

For images, CNNs usually perform better than feed forward nets even though CNNs have fewer parameters. Further, CNNs can be represented equivalently as feed forward nets. Why can't a feed forward net learn this equivalent weight matrix that the CNN learns? The answer probably lies in spatial connectivity. The convolution operator inherently depends only on the neighbours of a pixel and we know naturally that a pixel must always be considered in the context of its neighbours in order to characterise it appropriately. CNNs are forced into doing this while a feedforward net would have to learn this on its own and with a massive parameter search space, this is highly unlikely to happen.

### The Details of Convolution
#### Padding

Notice in the gif above that the output matrix is smaller than the input matrix. This happens because the filter is not allowed to exceed the bounds of the image and can result in incomplete usage of features located near the edge of the image and is a waste of valuable data. There are a few ways to overcome this by using padding. Padding refers to adding dummy pixels around the image so as to ensure that the convolution does not change the size of the output matrix. A few forms of padding are ’constant’, ’nearest’, ’mirror’ and ‘wrap’.

![padding](https://adeshpande3.github.io/assets/Pad.png)


#### Stride
In our example, we've been shifting the filter 1 unit right before every dot product. If we instead move it by 2 units, the size of the output would reduce by a factor of 2. The amount by which the filter moves is known as the stride and can be adjusted to decrease the amount of computation required in subsequent layers.


![stride1](https://adeshpande3.github.io/assets/Stride1.png)

Now, let us increase the stride to 2.

![stride2](https://adeshpande3.github.io/assets/Stride2.png)

#### Shift equivariance

Consider the case below where the same image is shifted and fed into some convolution operator.
![equivariance](https://i.imgur.com/CVwwPOe.png)
Notice that the output contains the same pattern of activations shifted by the same amount as the original image was shifted. This is known as translational equivariance. Convolutions are often misrepresented as being shift invariant but this is not the case since we can see that shifting the input does indeed change the output.

#### Handling Multiple Channels 

Earlier, we assumed that the input to our convolution was a grayscale image which can be represented as a single matrix, rather than an rgb image which must be represented as a stack of matrices. How does convolution handle this?
Instead of learning a filter matrix, we can learn 3 filter matrices (one for each channel) and stack them, creating a 3x3x3 filter. This is called a tensor.

![tensor](http://machinethink.net/images/vggnet-convolutional-neural-network-iphone/ConvolutionKernel@2x.png)

The tensor moves across the image in the same way as before, performing a dot product and giving a matrix output after passing over the entire image. Convolution is therefore a tensor to matrix operation.

#### Handling Multiple Filters

Given an image, we would like to extract as much information about it as possible. We might want to find all the different oriented edges separately (similar to the kirsch operator). This would involve using many filters at once. Since each filter outputs a matrix, it is fairly straightforward to stack these matrices giving a tensor with $f$ channels (where $f$ is the number of filters). 
Let's put all of this together and see what a convolution with multiple input and output channels looks like.

![multi in - multi out](http://machinethink.net/images/vggnet-convolutional-neural-network-iphone/ConvLayer@2x.png)

## Building a CNN
### Series Convolution
Now that we've seen how convolutions work, we can start to construct a neural network. The key idea here is to apply several convolutions in series so as to detect more and more nuanced features as we get deeper into the network. Why would this happen at all? We can show by a simple example that series convolutions can detect  shapes in an image.

Let's say we want to detect squares in our image. we could first use a kirsch filters to find the edges of the square first. The next set of convolutions can find whether these edges are perpendicular to each other and locate crossings. The next layer could convolve with a filter detecting 4 dots. This is a really simple example to show that features are heirarchical. We first detected edges, then combinations of edges and then shapes. Further layers can even detect textures.

### Pooling
A weakness of this method is that it isn't scale invariant. This wouldn't be able to detect squares of all sizes. 2 solutions to this are possible. The first solution would be to increase the stride of our filter so that spatially distant features in the image are brought closer together in the resulting feature map. As we go deeper into the network, feature maps would get smaller and smaller. The second way of going about this would be to perform a down sampling of the features maps at each stage. This is known as pooling. There are various methods of pooling like average and median, but the most common is called max pooling.

![pooling](http://ufldl.stanford.edu/tutorial/images/Pooling_schematic.gif)

Max pooling involves moving a box over contiguous areas of the feature map and selecting the max value in boxes. The strides of the boxes are generally equal to the size of the boxes. 2x2 max-pooling is very common in deep learning. It is also non-parametric so it doesn't need to learn any features to perform its operations, unlike filter convolutions. This is why most people prefer this method.

![Pooling operation](http://cs231n.github.io/assets/cnn/pool.jpeg)

In addition to bringing us closer to scale invariance, max pooling is robust to small shifts in the image, therefore we have also taken a  step in the direction of translation invariance.

### Fully Connected Layers
Convolutions suffer from an inherent bias: they only take into account the neighbourhood of a pixel. if we were for example trying to detect a car in a high resolution image, the network would have to perform multiple down samplings until a single filter is able to cover 2 active wheel detector neurons. We could instead use fully connected layers to take note of these distributed features.

### Putting it all together

![CNN](https://www.mathworks.com/content/mathworks/www/en/discovery/convolutional-neural-network/_jcr_content/mainParsys/image_copy.adapt.full.high.jpg/1517522275430.jpg)


# LENET5

It is the year 1994, and this is one of the very first convolutional neural networks, and what propelled the field of Deep Learning. 

This pioneering work by Yann LeCun was named LeNet5 after many previous successful iterations since they year 1988.

![alt text](https://cdn-images-1.medium.com/max/800/0*V1vb9SDnsU1eZQUy.jpg)



The LeNet5 architecture was fundamental, in particular the insight that image features are distributed across the entire image, and convolutions with learnable parameters are an effective way to extract similar features at multiple location with few parameters. 

At that time there was no GPU to help training, and even CPUs were slow.

Being able to save parameters and computation was a key advantage. Still not enough for deployment back then


LeNet5 features can be summarized as:

* convolutional neural network use sequence of 3 layers: convolution, pooling, non-linearity

* use convolution to extract spatial features

* non-linearity in the form of tanh or sigmoids (no ReLus back then)

* multi-layer neural network (MLP) as final classifier

* sparse connection matrix between layers to avoid large computational cost

## THE GAP

In the years from 1998 to 2010 neural network were in incubation.

Most people did not notice their increasing power, while many other researchers slowly progressed. 

Wasnot till 2010, when GPUs plus Deep training strategies allowed for their usage, first in the DAN CIRESAN NET.

# IMAGENET

 ImageNet Large Scale Visual Recognition Challenge (ILSVRC) evaluates algorithms for object detection and image classification at large scale. One high level motivation is to allow researchers to compare progress in detection across a wider variety of objects -- taking advantage of the quite expensive labeling effort. Another motivation is to measure the progress of computer vision for large scale image indexing for retrieval and annotation.

![alt text](https://raw.githubusercontent.com/TerrenceMiao/Data-Science/master/ImageNet%20Top%205%20Error%20Rate.jpeg)


# ALEXNET

In 2012, Alex Krizhevsky released AlexNet which was a deeper and much wider version of the LeNet and won by a large margin the difficult ImageNet competition.

Wide as in 20+% !

![alt text](https://cdn-images-1.medium.com/max/800/0*vsi8JJFV_O6Z34ks.png)

AlexNet scaled the insights of LeNet into a much larger neural network that could be used to learn much more complex objects and object hierarchies. The contribution of this work were:

* use of rectified linear units (ReLU) as non-linearities
* use of dropout technique (Hinton et all) to selectively ignore single neurons during training, a way to avoid overfitting of the model
* overlapping max pooling, avoiding the averaging effects of average pooling
* use of GPUs NVIDIA GTX 580 to reduce training time

At the time GPU offered a much larger number of cores than CPUs, and allowed 10x faster training time, which in turn allowed to use larger datasets and also bigger images.

The success of AlexNet started a small revolution. Convolutional neural network were now the workhorse of Deep Learning, which became the new name for “large neural networks that can now solve useful tasks”.

# VGG

The VGG networks from Oxford were the first to use much smaller 3×3 filters in each convolutional layers and also combined them as a sequence of convolutions.

This seems to be contrary to the principles of LeNet, where large convolutions were used to capture similar features in an image.

Instead of the 9×9 or 11×11 filters of AlexNet, filters started to become smaller, too dangerously close to the infamous 1×1 convolutions that LeNet wanted to avoid, at least on the first layers of the network. 

But the great advantage of VGG was the insight that multiple 3×3 convolution in sequence can emulate the effect of larger receptive fields, for examples 5×5 and 7×7. These ideas will be also used in more recent network architectures as Inception and ResNet.

![alt text](https://cdn-images-1.medium.com/max/800/0*HREIJ1hjF7z4y9Dd.jpg)

![alt text](https://qph.fs.quoracdn.net/main-qimg-ba81c87204be1a5d11d64a464bca39eb)

The VGG networks uses multiple 3×3 convolutional layers to represent complex features. 

Notice blocks 3, 4, 5 of VGG-E: 256×256 and 512×512 3×3 filters are used multiple times in sequence to extract more complex features and the combination of such features. 

This is effectively like having large 512×512 classifiers with 3 layers, which are convolutional! This obviously amounts to a massive number of parameters, and also learning power. 

But training of these network was difficult, and had to be split into smaller networks with layers added one by one. All this because of the lack of strong ways to regularize the model, or to somehow restrict the massive search space promoted by the large amount of parameters.

VGG used large feature sizes in many layers and thus inference was quite costly at run-time. Reducing the number of features, as done in Inception bottlenecks, will save some of the computational cost.



# GoogleNet and Inception
_Let the Games begin XD_

[Christian Szegedy](https://arxiv.org/abs/1409.4842) from Google begun a quest aimed at reducing the computational burden of deep neural networks, and devised the GoogLeNet the first Inception architecture.

By now, Fall 2014, deep learning models were becoming extermely useful in categorizing the content of images and video frames. 

Most skeptics had given in that Deep Learning and neural nets came back to stay this time. Given the usefulness of these techniques, the internet giants like Google were very interested in efficient and large deployments of architectures on their server farms.

Christian thought a lot about ways to reduce the computational burden of deep neural nets while obtaining state-of-art performance (on ImageNet, for example). Or be able to keep the computational cost the same, while offering improved performance.

He and his team came up with the Inception module:

![alt text](https://cdn-images-1.medium.com/max/800/0*CJZdXZULMr_on1Ao.jpg)



This at a first glance is basically the parallel combination of 1×1, 3×3, and 5×5 convolutional filters.

But the great insight of the inception module was the use of 1×1 convolutional blocks (NiN) to reduce the number of features before the expensive parallel blocks. This is commonly referred as “bottleneck”. T

GoogLeNet used a stem without inception modules as initial layers, and an average pooling plus softmax classifier similar to NiN. This classifier is also extremely low number of operations, compared to the ones of AlexNet and VGG. This also contributed to a very efficient network design.

![alt text](https://i.stack.imgur.com/Xqv0n.png)

## Why multiple softmaxes?

Deep architectures, and specifically GoogLeNet (22 layers) are in danger of the vanishing gradients problem during training (back-propagation algorithm). 

The engineers of GoogLeNet addressed this issue by adding classifiers in the intermediate layers as well, such that the final loss is a combination of the intermediate loss and the final loss. This is why you see a total of three loss layers, unlike the usual single layer as the last layer of the network.

![alt text](https://i.stack.imgur.com/U6O7Wl.png)

# Bottleneck layers

Was reducing the number of features, and thus operations, at each layer, so the inference time could be kept low. 

Before passing data to the expensive convolution modules, the number of features was reduce by, say, 4 times. This led to large savings in computational cost, and the success of this architecture.

Say you have 256 features coming in, and 256 coming out, and let’s say the Inception layer only performs 3×3 convolutions. 

That is 256×256 x 3×3 convolutions that have to be performed (589,000s multiply-accumulate, or MAC operations). 

Instead of doing this, we decide to reduce the number of features that will have to be convolved, say to 64 or 256/4. In this case, we first perform 256 -> 64 1×1 convolutions, then 64 convolution on all Inception branches, and then we use again a 1×1 convolution from 64 -> 256 features back again. The operations are now:

* 256×64 × 1×1 = 16,000s
* 64×64 × 3×3 = 36,000s
* 64×256 × 1×1 = 16,000s

For a total of about 70,000 versus the almost 600,000 we had before. Almost 10x less operations!

And although we are doing less operations, we are not losing generality in this layer. In fact the bottleneck layers have been proven to perform at state-of-art on the ImageNet dataset, for example, and will be also used in later architectures such as ResNet.

The reason for the success is that the input features are correlated, and thus redundancy can be removed by combining them appropriately with the 1×1 convolutions. Then, after convolution with a smaller number of features, they can be expanded again into meaningful combination for the next layer.

![alt text](https://raw.githubusercontent.com/ritchieng/machine-learning-nanodegree/master/deep_learning/convolutional_neural_nets/cn19.png)

Credits: Udacity

# INCEPTION V3 (AND V2)

In February 2015 Batch-normalized Inception was introduced as Inception V2. 

Batch-normalization computes the mean and standard-deviation of all feature maps at the output of a layer, and normalizes their responses with these values. This corresponds to “whitening” the data, and thus making all the neural maps have responses in the same range, and with zero mean. This helps training as the next layer does not have to learn offsets in the input data, and can focus on how to best combine features.

In December 2015 Google released a new version of the Inception modules and the corresponding architecture This article better explains the original GoogLeNet architecture, giving a lot more detail on the design choices. A list of the original ideas are:

* maximize information flow into the network, by carefully constructing networks that balance depth and width. Before each pooling, increase the feature maps.

* when depth is increased, the number of features, or width of the layer is also increased systematically

* use width increase at each layer to increase the combination of features before next layer

* use only 3×3 convolution, when possible, given that filter of 5×5 and 7×7 can be decomposed with multiple 3×3. 

![alt text](https://cdn-images-1.medium.com/max/800/0*Y9mKbwp1R8vAmT2L.jpg)

filters can also be decomposed by flattened convolutions into more complex modules:

![alt text](https://cdn-images-1.medium.com/max/800/0*rRv_N9rLYJnmq6jz.jpg)

Inception modules can also decrease the size of the data by provide pooling while performing the inception computation. 

This is basically identical to performing a convolution with strides in parallel with a simple pooling layer:

![alt text](https://cdn-images-1.medium.com/max/800/0*rxf30_SJRsbFIFCW.jpg)

Inception still uses a pooling layer plus softmax as final classifier.

Inception v3 in full glory:

![alt text](https://www.packtpub.com/graphics/9781786466587/graphics/image_08_007.jpg)


# Inception V3 on CIFAR 10

In [0]:
import cv2
import numpy as np

from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

nb_train_samples = 3000 # 3000 training samples
nb_valid_samples = 100 # 100 validation samples
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_dim_ordering() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

#load the data cifar10
X_train, Y_train, X_valid, Y_valid = load_cifar10_data(224,224)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=1000)
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# model.fit_generator(model,steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=X_valid, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

# RESNET

The revolution then came in December 2015, at about the same time as Inception v3. ResNet have a simple ideas: feed the output of two successive convolutional layer AND also bypass the input to the next layers!

![alt text](https://cdn-images-1.medium.com/max/800/0*0r0vS8myiqyOb79L.jpg)


![alt text](https://i.stack.imgur.com/ElFiI.png)

Similar approaches as inception though (bottlenecks)

Also, has variants- Resnet (34 layers), Resnet 50, Resnet 101 XD.

# Inception v4 or Inception_Resnet_v2

1.   List item
2.   List item


_There's no method to this madness_

And Christian and team are at it again with a new version of Inception.

The Inception module after the stem is rather similar to Inception V3:

![alt text](https://cdn-images-1.medium.com/max/800/0*SJ7DP_-0R1vdpVzv.jpg)


They also combined the Inception module with the ResNet module:

![alt text](https://cdn-images-1.medium.com/max/800/0*exGWbD4A0QKM2lU_.jpg)

# Covnet Accuracy

![alt text](https://cdn-images-1.medium.com/max/800/1*kBpEOy4fzLiFxRLjpxAX6A.png)

[Reference](https://arxiv.org/abs/1605.07678)



# Transfer Learning with Mobilenets

Clone the repo here: [link](https://github.com/iitmcvg/Fast-Image-Classification)